In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_style("whitegrid")
palette = sns.color_palette()

In [ ]:
sns.set_palette(palette)
palette

# Preparing the Dataframe

In [ ]:
if 'plots' in os.getcwd():
    print("cwd is correct")
else:
    MYDIR = ("plots")
    os.chdir("../")
    CHECK_FOLDER = os.path.isdir(MYDIR)
    ## If folder doesn't exist, then create it.
    if not CHECK_FOLDER:
        os.makedirs(MYDIR)
        print("created folder : ", MYDIR)

    else:
        print(MYDIR, "folder already exists.")

os.chdir("plots/")
print("current directroy is: ", os.getcwd())

In [ ]:
# check the data for NaN values or other errors
df_all = pd.read_csv(f"../../data_save/all_data.csv")

In [ ]:
pd.options.display.max_columns = 500 # this will set limit of columns to 500

In [ ]:
df_all.shape

In [ ]:
# drop all columns with NaN values
df_all = df_all.dropna(axis=1)
df_all.shape

In [ ]:
df_all['method'] = df_all['method'].astype('category')
df_all['method'] = df_all['method'].apply(lambda x: x.replace('_', ' ').title())

# remove all _ from the column names
df_all.columns = df_all.columns.str.replace('_', ' ')
# make all the columns tital like
df_all.columns = df_all.columns.str.title()

In [ ]:
df_all.columns

In [ ]:
df_all['M'] = df_all['M'].replace(['G'], 'AM')

In [ ]:
df_all['M'] = df_all['Method'].apply(lambda x: ''.join([str(i[0]) for i in x.split(' ')]))
df_end = df_all.drop_duplicates(subset=['Experiment Id'], keep='last').sort_values(by=['Experiment Id'])

In [ ]:
df_agent = df_all[df_all['Loss Type'] == 'Agent']
df_agent_end = df_agent.drop_duplicates(subset=['Experiment Id'], keep='last').sort_values(by=['Experiment Id'])

In [ ]:
df_all_success = df_all[df_all['Success'] == True]

# Worst Case Scenario

In [ ]:
df_all_success_four = df_all_success[df_all_success['N Bots'] == 4]

In [ ]:
wost_case = df_all_success_four.groupby(['M']).max()[['Frame Count', 'Delta Time']].sort_values(by='Frame Count')
with open(f"worst_case.txt", 'w') as f:
    f.write(wost_case.to_latex())
wost_case

# Quality Metric Table

In [ ]:
# quality_metric = df_agent_end.groupby(['M']).mean()
# quality_metric['Quality'] = quality_metric['Known Area'] / quality_metric['Total Distance Travelled']
# # show quality and method
# quality_metric = quality_metric.reset_index()[['M','Quality','Known Area','Total Distance Travelled']]
# # method_group[['M','Quality']]
# quality_metric = quality_metric.sort_values(by='Quality', ascending=False)
# quality_metric.reset_index()

# with open(f"success_metric.tex", 'w') as f:
#     # f.write(df_end.groupby(['Loss Type', 'Method','Success']).count()['Experiment Id'].to_latex())
#     # rotate the table 90 degrees so the index is now the columns
#     f.write(quality_metric.to_latex())
# quality_metric

# Failed Experiment Check

In [ ]:
# Total success count
print('Success Count: ', df_end['Success'].value_counts()[True])
# Total fail count
print('Fail Countsss: ', df_end['Success'].value_counts()[False])
# What is the overall success rate
print("Success rate is: ", df_end['Success'].value_counts()[True]/df_end['Success'].value_counts().sum())
total_success = df_end['Success'].value_counts()[True]
total_fail = df_end['Success'].value_counts()[False]
total_experiments = total_success + total_fail
print("Total number of experiments: ", total_experiments, "\n\n")


print(df_end[df_end['Success'] == False]['Loss Type'].value_counts())
print(df_end[df_end['Success'] == True]['Loss Type'].value_counts())
# print(df_end[df_end['Success'] == False]['Method'].value_counts())

In [ ]:
df_loss_unsuccessful = df_end[df_end['Success'] == False]

# df_loss_unsuccessful_disrepair = df_loss_unsuccessful[df_loss_unsuccessful['Loss Type'] == 'Disrepair']
# df_loss_unsuccessful_disrepair['N Bots'].value_counts()
# df_loss_unsuccessful_disrepair = df_loss_unsuccessful[df_loss_unsuccessful['Loss Type'] == 'Unrecoverable']
# df_loss_unsuccessful_disrepair['N Bots'].value_counts()

df_loss_unsuccessful['N Bots'].value_counts()
# sns.barplot(y="M", x="Success Ratio", hue="Loss Type", data=method_group.reset_index())

In [ ]:
loss_group = df_loss_unsuccessful.groupby(['M', 'N Bots']).count()['Experiment Id'].unstack()
loss_group

In [ ]:
# df_end['Success Ratio'] 
loss_group2 = df_loss_unsuccessful.groupby(['Loss Type', 'N Bots']).count()['Experiment Id'].unstack()
# loss_group2['Total'] = loss_group2[4]+loss_group2[8] + loss_group2[12]

with open(f"loss_count_vs_agents_size.tex", 'w') as f:
    f.write(loss_group2.to_latex())

loss_group2

In [ ]:
df_loss_unsuccessful_unrecoverable = df_loss_unsuccessful[df_loss_unsuccessful['Loss Type'] == 'Unrecoverable']

In [ ]:
sns.set_style("whitegrid")
# df_end_success = df_end_success.replace(to_replace="Goal Scenario", value="Helloooo")
sorted_x_data = df_loss_unsuccessful_unrecoverable.groupby("M")["Area Percent"].median().sort_values().index
grd = sns.boxplot(
    data=df_loss_unsuccessful_unrecoverable, 
    y="Area Percent", x="M", hue="N Bots", 
    showfliers=False,
    order=sorted_x_data,
)
# grd.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Strategies')
# make the legonf in two columns
grd.legend(ncol=3, title='Number of Robots')

grd.set_xticklabels(grd.get_xticklabels(), rotation=45, horizontalalignment='right')
# set x range 
# grd.set(ylim=(0, 310))
# set Y label
grd.set_ylabel("Final Area Percent")
grd.set_xlabel("Stratagies")
# grd.set(ylim=(0.9, 1))



plt.savefig(f"unsuccesssful_unrecoverable_analysis.png", bbox_inches='tight' ,dpi=300)

# Plots Start

In [ ]:
df_end_success = df_end[df_end['Success'] == True]

with open(f"methods.tex", 'w') as f:
    # f.write(df_end_success[['Method', "M"]].drop_duplicates().reset_index(drop=True).to_latex())
    # sort alphabetically
    f.write(df_end_success[['Method', "M"]].drop_duplicates().sort_values(by='Method').reset_index(drop=True).to_latex())

In [ ]:
result = df_end_success.groupby(["M"])['Frame Count'].aggregate(np.mean).reset_index().sort_values('Frame Count')

In [ ]:
sns.set_style('whitegrid', rc={'ytick.left': True})

grd = sns.barplot(
    data=df_end_success,
    x="N Bots",
    y="Frame Count",
    hue = "M",
    errorbar=("pi", 100),
    errwidth=1.5,
    capsize=.04,
    linewidth=0,
    alpha=.8,
    hue_order=result['M']
)
# grd.legend(ncol=5)
grd.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Strategies')

grd.set_ylabel("Time (seconds)")
grd.set_xlabel("Number of Robots")
grd.set_xticklabels(grd.get_xticklabels(), horizontalalignment='right')

plt.yticks(range(0, 1026, 25))
for ind, label in enumerate(grd.get_yticklabels()):
    if ind % 4 == 0:  # every 10th label is kept
        label.set_visible(True)
    else:
        label.set_visible(False)

plt.savefig(f"exploration_completion_time_barplot.png", bbox_inches='tight' ,dpi=300)

In [ ]:
sns.set_style('whitegrid', rc={'ytick.left': True})

grd = sns.lineplot(
    data=df_end_success,
    x="N Bots",
    y="Frame Count",
    hue = "M",
    alpha=.8,
    hue_order=result['M']
)
# grd.legend( ncol=2,)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Strategies')

grd.set_ylabel("Time (seconds)")
grd.set_xlabel("Number of Robots")

plt.xticks(range(4, 16, 4))
plt.yticks(range(0, 401, 25))
for ind, label in enumerate(grd.get_yticklabels()):
    if ind % 4 == 0:  # every 10th label is kept
        label.set_visible(True)
    else:
        label.set_visible(False)

plt.savefig(f"exploration_completion_time_lineplot.png", bbox_inches='tight' ,dpi=300)

In [ ]:
sns.set_style("whitegrid")
# df_end_success = df_end_success.replace(to_replace="Goal Scenario", value="Helloooo")

grd = sns.barplot(
    # data=df_end_success, 
    data = df_end_success[df_end_success['N Bots'] == 4],
    y="Frame Count", x="Min Room Size", hue="M", 
    # showfliers=False,
    errorbar=("pi", 100),
    errwidth=1.5,
    capsize=.04,
    # errcolor=".4",
    linewidth=0,

    # edgecolor=".5",
    alpha=.8,
    hue_order=result['M']
)
grd.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Strategies')
# make the legonf in two columns
# grd.legend( ncol=5,)


# set x range 
# grd.set(ylim=(0, 310))
# set Y label
grd.set_ylabel("Time (seconds)")
grd.set_xlabel("Minimum Room Size (meters)")
# grd.set(ylim=(0, 550))
# grd.set_xticklabels(rotation=30)
# plt.yticks(range(0, 310, 20))
plt.yticks(range(0, 1026, 25))
for ind, label in enumerate(grd.get_yticklabels()):
    if ind % 4 == 0:  # every 10th label is kept
        label.set_visible(True)
    else:
        label.set_visible(False)

# plt.grid(axis='y')
# plt.grid(color='gray', linestyle='dashed')
# need some cut before upload to overleaf
plt.savefig(f"room_density_vs_iteration_count_hue_strategies.png", bbox_inches='tight' ,dpi=300)

In [ ]:
df_all_success

In [ ]:
# grd = sns.lineplot(data=df_all_success_four, x = 'Frame Count',y = 'Replan Count', hue='M')
# grd.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Strategies')
# # plt.xticks(range(0, 1000, 100))
# # plt.ylim(0, 600)
# grd.set_ylabel("Replan Count")
# grd.set_xlabel("Time (seconds)")
# plt.savefig(f"time_vs_replan_count_hue_strategies_before_padding.png", bbox_inches='tight', dpi=300)

In [ ]:
# grd = sns.lineplot(data=df_all_success_four, x = 'Frame Count',y = 'Replan Count', hue='Min Room Size')
# grd.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Room Density')
# # plt.xticks(range(0, 1000, 100))
# # plt.ylim(0, 400)
# grd.set_ylabel("Replan Count")
# grd.set_xlabel("Time (seconds)")
# plt.savefig(f"time_vs_replan_count_hue_density_before_padding.png", bbox_inches='tight' ,dpi=300)

# PADDING DATA AND PLOTTING 2 FIGURES WITH PAD

In [ ]:
# for i,[method, group] in enumerate(df_all_success.groupby(['M'])):
#     print("i: ", i, "Method: ", method, "Group: ", group.shape)
#     group_counts = group.groupby('Experiment Id')['Replan Count'].apply(lambda x: x.iloc[-1])
#     unique =  set(group['Frame Count'].unique())
#     data2 = group.groupby("Experiment Id").agg({"Frame Count":lambda x: unique - set(x)}).explode("Frame Count").dropna().reset_index()

#     for j in group_counts.index:
#         data2.loc[data2['Experiment Id'] == j, "Replan Count"] = group_counts[j]
#     new_data = pd.concat([group, data2]).sort_values(["Experiment Id", "Frame Count"])
    
#     count_group = new_data.groupby(['Frame Count'])
#     x_data = count_group['Replan Count'].mean()
#     sns.lineplot(x_data, label=method)
#     # sns.set_ylabels("Replan Count")
# plt.xlabel("Time (seconds)")
# plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Strategies')    
# plt.savefig(f"time_vs_replan_count_hue_strategies_after_padding.png", bbox_inches='tight', dpi=300)

In [ ]:
# for i,[method, group] in enumerate(df_all_success.groupby(['Min Room Size'])):
#     print("i: ", i, "Method: ", method, "Group: ", group.shape)
#     group_counts = group.groupby('Experiment Id')['Replan Count'].apply(lambda x: x.iloc[-1])
#     unique =  set(group['Frame Count'].unique())
#     data2 = group.groupby("Experiment Id").agg({"Frame Count":lambda x: unique - set(x)}).explode("Frame Count").dropna().reset_index()

#     for j in group_counts.index:
#         data2.loc[data2['Experiment Id'] == j, "Replan Count"] = group_counts[j]
#     new_data = pd.concat([group, data2]).sort_values(["Experiment Id", "Frame Count"])
    
#     count_group = new_data.groupby(['Frame Count'])
#     x_data = count_group['Replan Count'].mean()
#     sns.lineplot(x_data, label=method)
#     # sns.set_ylabels("Replan Count")
# plt.xlabel("Time (seconds)")
# plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Strategies')    
# plt.savefig(f"time_vs_replan_count_hue_density_after_padding.png", bbox_inches='tight', dpi=300)

# PADDING FINISH

In [ ]:
df_end_success['Goal Scenario'].unique()

In [ ]:
df_end_success['Goal Scenario'] = df_end_success['Goal Scenario'].replace(['goal-Center_Start'], 'Center of Map')
df_end_success['Goal Scenario'] = df_end_success['Goal Scenario'].replace(['goal-Distributed_Start'], 'Equal Spread')
df_end_success['Goal Scenario'] = df_end_success['Goal Scenario'].replace(['goal-Edge_Start'], 'Edges of Map')
df_end_success['Goal Scenario'] = df_end_success['Goal Scenario'].replace(['goal-Rand_Start'], 'Random Location')
df_end_success['Goal Scenario'] = df_end_success['Goal Scenario'].replace(['goal-Top_Left_Start'], 'Top Left Corner')


In [ ]:
df_end_success['Goal Scenario'].unique()

In [ ]:
palet2 = sns.color_palette("bright", 10)
paletinv = palet2.reverse()
paletinv


In [ ]:
result22 = df_end_success.groupby(["Goal Scenario"])['Frame Count'].aggregate(np.mean).reset_index().sort_values('Frame Count')

In [ ]:
sns.set_style('whitegrid', rc={'ytick.left': True})

grd = sns.barplot(
    data = df_end_success[df_end_success['N Bots'] == 4],
    y="Frame Count", x="M", hue="Goal Scenario", 
    palette = palet2,
    
    errorbar=("pi", 100),
    errwidth=1.5,
    capsize=.04,
    # errcolor=".4",
    linewidth=0,
    
    order=result['M'],
    hue_order = result22['Goal Scenario']
    
    
)
grd.legend(ncol=3, loc='upper center', bbox_to_anchor=(0.5, 1.2), title='Distribution Scenarios')
# plt.legend(loc='upper left', title='Distribution Scenarios')
# grd.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Distribution Scenarios')
grd.set_ylabel("Time (seconds)")
grd.set_xlabel("Strategies")
grd.set_xticklabels(grd.get_xticklabels(), horizontalalignment='right', rotation=30),

# plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Strategies')

plt.yticks(range(0, 1026, 25))
for ind, label in enumerate(grd.get_yticklabels()):
    if ind % 4 == 0:  # every 10th label is kept
        label.set_visible(True)
    else:
        label.set_visible(False)

plt.savefig(f"distribution_vs_segmentation.png", bbox_inches='tight' ,dpi=300)

In [ ]:
Area_max = df_all['Rows'].iloc[0] * df_all['Cols'].iloc[0]
Area_max

In [ ]:
df_end_success_four = df_end_success[df_end_success['N Bots'] == 4]

In [ ]:
# calculate Quality df_all for each all the data
# Quality = Area_max/Total Distance Traveled
df_end_success_four['Quality'] = Area_max/df_end_success_four['Total Distance Travelled']
df_end_success_four['Quality'].describe()

In [ ]:
# Sort the x-axis data based on "Quality"
sorted_x_data = df_end_success_four.groupby("M")["Quality"].median().sort_values().index

grd = sns.boxplot(
    data=df_end_success_four, 
    y="Quality", 
    hue="Loss Type", 
    x="M", 
    order=sorted_x_data,  # Specify the sorted x-axis data
    showfliers=False,
    # alpha=.8,
    hue_order=['Disrepair', 'Agent', 'Unrecoverable']
)

# grd.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Loss Types')
grd.set_xticklabels(grd.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.yticks(range(0, 13, 1))

# rename the x axis 
grd.set(xlabel='Strategies')
grd.set(ylabel='Quality Ratio: Area/Distance')

plt.savefig(f"distance_cost_metric.png", dpi=300, bbox_inches='tight')
plt.show()  # Optionally display the plot